In [1]:
#Part1
# load the libraries needed
!pip install pulp

In [2]:
from pulp import *
import pandas as pd

In [3]:
# load the diet data
df = pd.read_excel(open('diet.xls','rb'),sheet_name='Sheet1')

In [4]:
# look at the data 
df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [5]:
# clean data - take first 64 rows not including bottom data
data = df[0:64]

In [6]:
# convert to list "list within a list"
data = data.values.tolist()

In [7]:
# create master foods dictionary
foods = [x[0] for x in data]
calories = dict([(x[0], float(x[3])) for x in data])
cholesterol = dict([(x[0], float(x[4])) for x in data])
totalFat = dict([(x[0], float(x[5])) for x in data])
sodium = dict([(x[0], float(x[6])) for x in data])
carbs = dict([(x[0], float(x[7])) for x in data])
fiber = dict([(x[0], float(x[8])) for x in data])
protien = dict([(x[0], float(x[9])) for x in data])
vitaminA = dict([(x[0], float(x[10])) for x in data])
vitaminC = dict([(x[0], float(x[11])) for x in data])
calcium = dict([(x[0], float(x[12])) for x in data])
iron = dict([(x[0], float(x[13])) for x in data])

In [8]:
# create list for mins and maxes (all foods)
amin = [1500, 30, 20, 800, 130, 125, 60, 1000, 400, 700, 10]
amax = [2500, 240, 70, 2000, 450, 250, 100, 10000, 5000, 1500, 40]

In [9]:
# append collection of contraints for each column 
B = []
for j in range(0,11):
    B.append(dict([(x[0], float(x[j+3])) for x in data]))

In [10]:
# define the cost dictionary
cost = dict([(x[0], float(x[1])) for x in data])

In [11]:
# create the optimization problem framework - minimization problem
problem1 = LpProblem('PuLPTutorial', LpMinimize)

In [12]:
# define the variables - continous
foodVars = LpVariable.dicts("foods", foods,0)

In [13]:
# dictionary of lp variables 
x = LpVariable.dicts("x", foods, 0)

In [14]:
# define the objective function
problem1 += lpSum([cost[f] * foodVars[f] for f in foods])

In [15]:
# add constraints for all foods
for i in range(0,11):
    dot_B_x = pulp.lpSum([B[i][j] * foodVars[j] for j in foods])
    condition1 = amin[i] <= + dot_B_x
    problem1 += condition1

In [16]:
for i in range(0,11):
    dot_B_x = pulp.lpSum([B[i][j] * foodVars[j] for j in foods])
    condition2 = amax[i] >= + dot_B_x
    problem1 += condition2

In [17]:
# solve the optimization problem!
problem1.solve()

1

In [18]:
# print the foods of the optimal diet
print('Optimization Solution:')
for var in problem1.variables():
    if var.varValue > 0:
        if str(var).find('Chosen'):
            print(str(var.varValue) + " units of " + str(var))

Optimization Solution:
52.64371 units of foods_Celery,_Raw
0.25960653 units of foods_Frozen_Broccoli
63.988506 units of foods_Lettuce,Iceberg,Raw
2.2929389 units of foods_Oranges
0.14184397 units of foods_Poached_Eggs
13.869322 units of foods_Popcorn,Air_Popped


In [19]:
# print the costs of the optimal diet             
print("Total cost of food = $%.2f" % value(problem1.objective))

Total cost of food = $4.34


In [20]:
# create the optimization problem framework - minimization problem
problem2 = LpProblem('PuLPTutorial', LpMinimize)

In [21]:
# define the variables - binary
chosenVars = LpVariable.dicts("Chosen",foods,0,1,"Binary")

In [22]:
# define the objective function
problem2 += lpSum([cost[f] * foodVars[f] for f in foods])

In [23]:
# add contraints amount greater than .1 or less than large amount - if chosen
for f in foods:
    problem2 += foodVars[f] <= 10000 * chosenVars[f]
    problem2 += foodVars[f] >= .1 * chosenVars[f]

In [24]:
# add constraints for all foods
for i in range(0,11):
    dot_B_x = pulp.lpSum([B[i][j] * foodVars[j] for j in foods])
    condition1 = amin[i] <= + dot_B_x
    problem2 += condition1

In [25]:
for i in range(0,11):
    dot_B_x = pulp.lpSum([B[i][j] * foodVars[j] for j in foods])
    condition2 = amax[i] >= + dot_B_x
    problem2 += condition2

In [26]:
# add contraints to eat at most one of a group of foods    
problem2 += chosenVars['Frozen Broccoli'] + \
chosenVars['Celery, Raw'] <= 1, 'At most one Broccoli / Celery'

In [27]:
# add contraints that says we require to eat as least 1 from group of food
problem2 += chosenVars['Roasted Chicken'] + chosenVars['Poached Eggs'] + \
  chosenVars['Scrambled Eggs'] + chosenVars['Frankfurter, Beef'] + \
  chosenVars['Kielbasa,Prk'] + chosenVars['Hamburger W/Toppings'] + \
  chosenVars['Hotdog, Plain'] + chosenVars['Pork'] + \
  chosenVars['Bologna,Turkey'] + chosenVars['Ham,Sliced,Extralean'] + \
  chosenVars['White Tuna in Water'] \
  >= 3, 'At least three proteins'

In [28]:
# solve the optimization problem!
problem2.solve()

1

In [29]:
# print the foods of the optimal diet
print('Optimization Solution:')
for var in problem2.variables():
    if var.varValue > 0:
        if str(var).find('Chosen'):
            print(str(var.varValue) + " units of " + str(var))

Optimization Solution:
42.399358 units of foods_Celery,_Raw
0.1 units of foods_Kielbasa,Prk
82.802586 units of foods_Lettuce,Iceberg,Raw
3.0771841 units of foods_Oranges
1.9429716 units of foods_Peanut_Butter
0.1 units of foods_Poached_Eggs
13.223294 units of foods_Popcorn,Air_Popped
0.1 units of foods_Scrambled_Eggs


In [30]:
# print the costs of the optimal diet             
print("Total cost of food = $%.2f" % value(problem2.objective))

Total cost of food = $4.51
